In [84]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
prompt_template = HumanMessagePromptTemplate.from_template("""
Rewrite into a valid JSON. The output should only contain the fixed JSON so that it can be loaded using json.loads
                                                           
Input invalid JSON:
{{
    "name": "John Doe",
    "age": 30,
    "isStudent": false
    "address": {{
        "street": "123 Main St",
        "city": "Anytown",
        "state": "CA",
    }}
    "hobbies": ["reading", "swimming", "cycling"]
}}
Fixed JSON:
{{
    "name": "John Doe",
    "age": 30,
    "isStudent": false,
    "address": {{
        "street": "123 Main St",
        "city": "Anytown",
        "state": "CA"
    }},
    "hobbies": ["reading", "swimming", "cycling"]
}}


Input invalid JSON:
{{
    "statement": "The Earth is also known as "Terra" "
}}
Fixed JSON:
{{
    "statement": "The Earth is also known as 'Terra'"
}}

Input invalid JSON:
{input}

Fixed JSON:"""
)

In [2]:
prompt_template2 = """
Rewrite into a valid JSON. The output JSON should be throw the error {err} when loaded using json.loads

Input:
{{
    "name": "John Doe",
    "age": 30,
    "isStudent": false
    "address": {{
        "street": "123 Main St",
        "city": "Anytown",
        "state": "CA",
    }}
    "hobbies": ["reading", "swimming", "cycling"]
}}
Ouput:
{{
    "name": "John Doe",
    "age": 30,
    "isStudent": false,
    "address": {{
        "street": "123 Main St",
        "city": "Anytown",
        "state": "CA"
    }},
    "hobbies": ["reading", "swimming", "cycling"]
}}


Input:
{{
    "statement": "The Earth is also known as "Terra" "
}}
Output:
{{
    "statement": "The Earth is also known as 'Terra'"
}}

Input:
{input}

Ouput:""" 

In [ ]:
from langchain.llms import HuggingFaceHub, Bedrock
from ragas.llms import LangchainLLM
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import BedrockChat
import json

In [15]:
def bedrock_llm():
    config = {
        "credentials_profile_name": "default",  # E.g "default"
        "region_name": "us-east-1",  # E.g. "us-east-1"
        "model_id": "anthropic.claude-v2",  # E.g "anthropic.claude-v2"
        "model_kwargs": {"temperature": 0.4},
    }
    #prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    bedrock_model = BedrockChat(
        credentials_profile_name=config["credentials_profile_name"],
        region_name=config["region_name"],
        endpoint_url=f"https://bedrock-runtime.{config['region_name']}.amazonaws.com",
        model_id=config["model_id"],
        model_kwargs=config["model_kwargs"],
    )
    # wrapper around bedrock_model
    ragas_bedrock_model = LangchainLLM(bedrock_model)
    return ragas_bedrock_model

In [16]:
def hf_llm(repo_id, prompt):
    repo_id = repo_id
    hf_llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 64,"return_full_text": False}
    )
    # wrapper around bedrock_model
    llm_chain = LLMChain(prompt=prompt, llm=hf_llm)
    return llm_chain

In [80]:
import warnings
def generate_answer(llm, prompt, text):
    human_prompt = prompt.format(input=text)
    prompt = ChatPromptTemplate.from_messages([human_prompt])
    results = llm.generate(prompts=[prompt])
    return results.generations[0][0].text.strip()


def load_as_json(llm, prompt, text, type):
    """
    validate and return given text as json
    """

    try:
        json_val = json.loads(text)
        return json_val, 0, []
    except Exception as err:
        retry = 0
        generations = []
        while retry<3:
            if type=='bedrock':
                generated_text=generate_answer(llm, prompt, text)
                generations.append(generated_text)
            else:
                generated_text= llm.run(text)
            try:
                return json.loads(generated_text), retry, generations
            except ValueError as e:
                retry+=1
        warnings.warn(f"Invalid json: {err}")
        print(err)
        return {}, 0, generations

In [89]:
with open('./test_json_correction.json') as f:
    data = json.load(f)
    

In [85]:
final = []
llm = bedrock_llm()
for i in data['inputs'][:3]:
    res, retry, generations = load_as_json(llm, prompt_template, i, 'bedrock')
    final.append([res, retry, generations])

us-east-1
us-east-1
us-east-1


/var/folders/5k/ljq_dfpn523_hcl4l6s9v1vc0000gn/T/ipykernel_46867/2928285411.py:30: UserWarning: Invalid json: Expecting property name enclosed in double quotes: line 1 column 23 (char 22)
  warnings.warn(f"Invalid json: {err}")


Expecting property name enclosed in double quotes: line 1 column 23 (char 22)
us-east-1
us-east-1
us-east-1


/var/folders/5k/ljq_dfpn523_hcl4l6s9v1vc0000gn/T/ipykernel_46867/2928285411.py:30: UserWarning: Invalid json: Expecting property name enclosed in double quotes: line 1 column 43 (char 42)
  warnings.warn(f"Invalid json: {err}")


Expecting property name enclosed in double quotes: line 1 column 43 (char 42)
us-east-1
us-east-1
us-east-1
Expecting ',' delimiter: line 1 column 42 (char 41)


/var/folders/5k/ljq_dfpn523_hcl4l6s9v1vc0000gn/T/ipykernel_46867/2928285411.py:30: UserWarning: Invalid json: Expecting ',' delimiter: line 1 column 42 (char 41)
  warnings.warn(f"Invalid json: {err}")


In [86]:
final

[[{},
  0,
  ['Here is the fixed valid JSON:\n\n{\n  "name": "John Doe",\n  "age": 30,\n  "city": "New York"\n}',
   'Here is the fixed valid JSON:\n\n{\n  "name": "John Doe",\n  "age": 30,\n  "city": "New York"\n}',
   'Here is the fixed valid JSON:\n\n{\n  "name": "John Doe",\n  "age": 30,\n  "city": "New York"\n}']],
 [{},
  0,
  ['Here is the fixed valid JSON for the given invalid JSON inputs:\n\n{\n    "name": "John Doe", \n    "age": 30,\n    "isStudent": false,\n    "address": {\n        "street": "123 Main St",\n        "city": "Anytown",\n        "state": "CA"\n    },\n    "hobbies": ["reading", "swimming", "cycling"]\n}\n\n{\n    "statement": "The Earth is also known as \'Terra\'"\n}\n\n[\n    {\n        "item1": "value1",\n        "item2": "value2"\n    }\n]',
   'Here is the fixed valid JSON for the given invalid JSON inputs:\n\n{\n    "name": "John Doe", \n    "age": 30,\n    "isStudent": false,\n    "address": {\n        "street": "123 Main St",\n        "city": "Anytown"